# Interacting with Kinetica and the SQLAssist LLM

In [13]:
pip install pycatch22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycatch22: filename=pycatch22-0.4.4-cp310-cp310-macosx_14_0_arm64.whl size=53012 sha256=8eae7600eca0d5d83f9277df0151a9eb3c38bb0b84a1d575b54c08a0c0f8a43b
  Stored in directory: /Users/Nima/Library/Caches/pip/wheels/10/67/84/cdce1a956aa218fd5ce5b5fa6773219f42780b1fac77889c57
Successfully built pycatch22

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Connect to Kinetica and the LLM

In [7]:
from dotenv import load_dotenv
import importlib
import kinetica.kinetica_ctx as ctx
from kinetica.kinetica_ctx import KineticaTableDefinition, KineticaSamplesDefinition, KineticaContextBuilder
import os

load_dotenv() 
user     = '<YOUR_USERNAME>'
password = '<YOUR_PASSWORD>'
host     = 'https://demo72.kinetica.com/_gpudb/'

importlib.reload(ctx)

<module 'kinetica.kinetica_ctx' from '/Users/Nima/Desktop/kinetica_demos/kinetica/kinetica_ctx.py'>

### Set up the context

In [8]:
from gpudb import GPUdb
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.kinetica import ChatKinetica, KineticaSqlOutputParser, KineticaSqlResponse, KineticaUtil

# Set the SQL context to use
kinetica_ctx: str = 'raceday.raceday_ui_ctxt'

# create the Kinetica connection
kdbc: GPUdb = KineticaUtil.create_kdbc(url=host, user=user, passwd=password)

In [9]:
# create the Kinetica LLM
kinetica_llm = ChatKinetica(kdbc=kdbc)

# load the context from the database
ctx_messages = kinetica_llm.load_messages_from_context(kinetica_ctx)

# Add the input prompt. This is where input question will be substituted.
ctx_messages.append(("human", "{input}"))

# Create the prompt template.
prompt_template = ChatPromptTemplate.from_messages(ctx_messages)
prompt_template.pretty_print()

# create the chain. 
# note: The KineticaSqlOutputParser will execute the SQL statement and is optional.
chain = prompt_template | kinetica_llm | KineticaSqlOutputParser(kdbc=kdbc)
# for error handling purposes, we are making it easy to troubleshoot the generated SQL
chain_sql_only = prompt_template | kinetica_llm

================================ System Message ================================

CREATE TABLE raceday.video_join1 AS
(
   IsRaceOn INTEGER,
   timestamp DATETIME  COMMENT 'this is the timestamp column',
   EngineMaxRpm REAL  COMMENT 'this is the maximum engine RPM',
   EngineIdleRpm REAL,
   CurrentEngineRpm REAL,
   AccelerationX REAL,
   AccelerationY REAL,
   AccelerationZ REAL,
   VelocityX REAL,
   VelocityY REAL,
   VelocityZ REAL,
   AngularVelocityX REAL,
   AngularVelocityY REAL,
   AngularVelocityZ REAL,
   Yaw REAL,
   Pitch REAL,
   Roll REAL,
   NormSuspensionTravelFl REAL,
   NormSuspensionTravelFr REAL,
   NormSuspensionTravelRl REAL,
   NormSuspensionTravelRr REAL,
   TireSlipRatioFl REAL,
   TireSlipRatioFr REAL,
   TireSlipRatioRl REAL,
   TireSlipRatioRr REAL,
   WheelRotationSpeedFl REAL,
   WheelRotationSpeedFr REAL,
   WheelRotationSpeedRl REAL,
   WheelRotationSpeedRr REAL,
   WheelOnRumbleStripFl INTEGER,
   WheelOnRumbleStripFr INTEGER,
   WheelOnRumbleStripRl

### A simple question

In [10]:
from IPython.display import display, HTML

question = {"input": "show me 5 videos where a crash is likely to occur"}


try:
    
    
    response: KineticaSqlResponse = chain.invoke(question)
    df = response.dataframe
    
    
    
    if df is None:
        print('No records returned\n', response.sql)
    else:
        display(HTML(response.dataframe.to_html(index=False)))
        
except:
    response = chain_sql_only.invoke(question)
    print('Got an error from the SQL query:\n',response.content)

ts,videopath
2023-10-30 19:27:46,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4
2023-10-30 19:27:51,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-16.mp4
2023-10-30 19:28:46,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-27.mp4
2023-10-30 19:29:26,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-35.mp4
2023-10-30 19:42:25,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-50.mp4


### Based off of thresholds, we get reasonable results, but we can do better with vector similarity search
<video src="https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4" width="500" controls></video>|<video src="https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-27.mp4" width="500" controls></video>

# Vector Similarity Search
First, we need to generate our vector embeddings.  For this exercise, we will be using the [catch22 library](https://time-series-features.gitbook.io/catch22-features/), which is, as you'd expect, a collection of 22 different features specifically focused on time-series data.

In [14]:
import pycatch22
import gpudb
import json
import math

def c22(ts_data: list) -> list:
    timeseries = [pycatch22.CO_f1ecac(ts_data),
                  pycatch22.CO_trev_1_num(ts_data),
                  pycatch22.CO_FirstMin_ac(ts_data),
                  pycatch22.CO_HistogramAMI_even_2_5(ts_data),
                  pycatch22.DN_Mean(ts_data),
                  pycatch22.DN_Spread_Std(ts_data),
                  pycatch22.DN_HistogramMode_5(ts_data),
                  pycatch22.DN_HistogramMode_10(ts_data),
                  pycatch22.DN_OutlierInclude_n_001_mdrmd(ts_data),
                  pycatch22.SB_BinaryStats_diff_longstretch0(ts_data),
                  pycatch22.SB_BinaryStats_mean_longstretch1(ts_data),
                  pycatch22.SB_MotifThree_quantile_hh(ts_data),
                  pycatch22.SB_TransitionMatrix_3ac_sumdiagcov(ts_data),
                  pycatch22.SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1(ts_data),
                  pycatch22.SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1(ts_data),
                  pycatch22.SP_Summaries_welch_rect_area_5_1(ts_data),
                  pycatch22.SP_Summaries_welch_rect_centroid(ts_data),
                  pycatch22.FC_LocalSimple_mean1_tauresrat(ts_data),
                  pycatch22.FC_LocalSimple_mean3_stderr(ts_data),
                  pycatch22.IN_AutoMutualInfoStats_40_gaussian_fmmi(ts_data),
                  pycatch22.MD_hrv_classic_pnn40(ts_data),
                  pycatch22.PD_PeriodicityWang_th0_01(ts_data)]

    return timeseries

### Connect to Kinetica

### Create the vector table schema

In [18]:
# we are going to truncate the table
if kdbc.has_table("raceday.vectors")["table_exists"]:
    kdbc.clear_table(table_name="raceday.vectors")

schema = [
    ["ts_bkt"             , "string", "datetime"],
    ["TireSlipRatioFl_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioFr_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioRl_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioRr_Vec", "bytes" , "vector(22)"],
    ["AccelerationX_Vec"  , "bytes" , "vector(22)"],
    ["AccelerationY_Vec"  , "bytes" , "vector(22)"],
    ["AccelerationZ_Vec"  , "bytes" , "vector(22)"],
    ["VelocityX_Vec"      , "bytes" , "vector(22)"],
    ["VelocityY_Vec"      , "bytes" , "vector(22)"],
    ["VelocityZ_Vec"      , "bytes" , "vector(22)"],
    ["CombinedRace_Vec"   , "bytes" , "vector(220)"]
]

tableObj = gpudb.GPUdbTable(
    _type=schema,
    name="raceday.vectors",
    use_multihead_io=False,
    multihead_ingest_batch_size=5000,
    db=kdbc
)

### Now that we have created our vector table, we need to loop through all the time-series data, convert to our 22-dimensional vector embedding space, then insert into Kinetica

In [20]:
has_more = True
offset = 0
while has_more:
    result = None
    try:
        result = kdbc.execute_sql('''select
       time_bucket(interval 5 second, timestamp) as ts_bkt,
       TireSlipRatioFl,
       TireSlipRatioFr,
       TireSlipRatioRl,
       TireSlipRatioRr,
       AccelerationX,
       AccelerationY,
       AccelerationZ,
       VelocityX,
       VelocityY,
       VelocityZ
    from
        raceday.dash1''',
                                encoding='json',
                                offset=offset)

    except gpudb.GPUdbException as gpudberror:
        print(str(gpudberror))
        
    if result is not None:
        j = json.loads(result.json_encoded_response)
        flat_result = {}
        count = 0

        has_more = result.has_more_records
        offset += len(j['column_1'])
        
        for count in range(0, len(j['column_1'])):
            if j['column_1'][count] not in flat_result:
                flat_result[j['column_1'][count]] = {
                    'TireSlipRatioFl_Vec': [],
                    'TireSlipRatioFr_Vec': [],
                    'TireSlipRatioRl_Vec': [],
                    'TireSlipRatioRr_Vec': [],
                    'AccelerationX_Vec':   [],
                    'AccelerationY_Vec':   [],
                    'AccelerationZ_Vec':   [],
                    'VelocityX_Vec':       [],
                    'VelocityY_Vec':       [],
                    'VelocityZ_Vec':       [],
                    'CombinedRace_Vec':    []
                }

            flat_result[j['column_1'][count]]['TireSlipRatioFl_Vec'].append(j['column_2'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioFr_Vec'].append(j['column_3'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioRl_Vec'].append(j['column_4'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioRr_Vec'].append(j['column_5'][count])
            flat_result[j['column_1'][count]]['AccelerationX_Vec'].append(j['column_6'][count])
            flat_result[j['column_1'][count]]['AccelerationY_Vec'].append(j['column_7'][count])
            flat_result[j['column_1'][count]]['AccelerationZ_Vec'].append(j['column_8'][count])
            flat_result[j['column_1'][count]]['VelocityX_Vec'].append(j['column_9'][count])
            flat_result[j['column_1'][count]]['VelocityY_Vec'].append(j['column_10'][count])
            flat_result[j['column_1'][count]]['VelocityZ_Vec'].append(j['column_11'][count])
            
        for key, value in flat_result.items():
            for k2, v2 in value.items():
                if k2 != 'CombinedRace_Vec':
                    vec = c22(v2)
                    vec = [0 if math.isnan(x) else x for x in vec]
                    flat_result[key][k2] = vec
                    flat_result[key]['CombinedRace_Vec'].extend(vec)
                    
        for key, value in flat_result.items():
            json_record = json.dumps({'ts_bkt':              key,
                                      'TireSlipRatioFl_Vec': value['TireSlipRatioFl_Vec'],
                                      'TireSlipRatioFr_Vec': value['TireSlipRatioFr_Vec'],
                                      'TireSlipRatioRl_Vec': value['TireSlipRatioRl_Vec'],
                                      'TireSlipRatioRr_Vec': value['TireSlipRatioRr_Vec'],
                                      'AccelerationX_Vec':   value['AccelerationX_Vec'],
                                      'AccelerationY_Vec':   value['AccelerationY_Vec'],
                                      'AccelerationZ_Vec':   value['AccelerationZ_Vec'],
                                      'VelocityX_Vec':       value['VelocityX_Vec'],
                                      'VelocityY_Vec':       value['VelocityY_Vec'],
                                      'VelocityZ_Vec':       value['VelocityZ_Vec'],
                                      'CombinedRace_Vec':    value['CombinedRace_Vec']
                                      })
            response = kdbc.insert_records_from_json(
                table_name='raceday.vectors',
                json_records=json_record)
tableObj.flush_data_to_server()

### Using SQLAssist to use vector similarity search

In [22]:
# Here you must ask a question relevant to the LLM context provided in the prompt template.
question = {"input": 'find videos similar to https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4'}
try:
    response: KineticaSqlResponse = chain.invoke(question)
    df = response.dataframe
    if df is None:
        print('No records returned\n', response.sql)
    else:
        display(HTML(response.dataframe.to_html(index=False)))
        
except:
    response = chain_sql_only.invoke(question)
    print('Got an error from the SQL query:\n',response.content)

ts_bkt,videopath,similarity
2023-10-30 19:27:50,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4,0.0
2023-10-30 19:42:20,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-49.mp4,0.028669
2023-10-30 19:42:00,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-45.mp4,0.046747
2023-10-30 19:24:00,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-1-03.mp4,0.069665
2023-10-30 19:25:10,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-1-17.mp4,0.071257


### Here is the video that we want to use as our search vector
<video src="https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4" width="500" controls></video>

### Here is the most similar video based off of cosine distance
<video src="https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-49.mp4" width="500" controls></video>

# Interacting with Nemo
Now we want to do something interesting, so instead of just converting natural language to SQL, lets get two LLM's talking to each other

In [10]:
import importlib
import kinetica.kineai
from kinetica.kineai import SqlAssistLLM

importlib.reload(kinetica.kineai)
kineticallm = kinetica.kineai.KineticaLLM(kinetica_ctx)

### Create the context object

In [11]:
system = """ KineticAI is a cheerful AI assistant for engaging in a conversation between an LLM using the Nemo framework and the Kinetica LLM.  The Kinetica
LLM is designed to translate natural language questions into SQL queries. 

In addition to responding with  natural language it is able to ask questions to a database AI named SqlAssist that can query and summarize the logs. 
If it responds with a "KineticaLLM |  question" where question is sent to the SqlAssist AI. The SqlAssist AI will respond with an answer 
to the question in JSON format to the question made to SqlAssist by KineticAI.

when presented with a question, you should prefix your response with "KineticaLLM |  "
if a sentence ends in a "?", you should prefix your response with "KineticaLLM |  "

Consider the following example where a user asks KineticAI a question and KineticAI asks a followup question to SqlAssist. KineticAI uses the response from 
SqlAssist to answer the user's question.

user: what is the weather like today?
assistant: KineticaLLM |  what is the weather like today?
user: KineticaLLM |  [{"EXPR_0": 5.4}]
assistant: The answer is 5.4
"""

context0 = [dict(role="system", content=system),
            dict(role="user", content="what is the weather like today?"),
            dict(role="assistant", content="KineticaLLM |  what is the weather like today?"),
            dict(role="user", content="how many rows of data are you storing?"),
            dict(role="assistant", content="KineticaLLM |  how many rows of data are you storing?"),
            dict(role="user", content="what is the average number of telemetry rows per 5 second increment?"),
            dict(role="assistant", content="KineticaLLM |  what is the average number of telemetry rows per 5 second increment?"),
            dict(role="user", content="find me videos"),
            dict(role="assistant", content="KineticaLLM |  find me videos")]

# samples


In [12]:
question = 'what is the average velocity along the X axis?'
response = kineticallm.chat(context0, question)

INFO:NemoChatLLM:user: KineticaLLM |  [{"EXPR_0": 1.077429120258083}] (tokens: 803/3293)
INFO:NemoChatLLM:assistant: KineticaLLM |  [{"EXPR_0": 1.0774291 (tokens: 900/3196)
